# Import libraries


In [1]:
import os
import sys
import subprocess
import tqdm
from typing import List, Tuple
import shlex

# Verify The Total TXL Can Parse
function named `parse_unparse_contracts` is used to parse and unparse contracts using TXL.
The function takes two arguments:
- `txl`: the path to the TXL file
- `paths`: a list of paths to the contracts to parse
The function returns two lists:
- `outputs`: a list of paths to the contracts that were parsed
- `errors`: a list of paths to the contracts that were not parsed

for errors is a list of paths to the contracts that were not parsed.
for outputs is a list of paths to the contracts that were parsed.
$$TXL_{errors} = |errors|$$
$$TXL_{outputs} = |outputs|$$
$$TXL_{total} = TXL_{errors} + TXL_{outputs}$$
$$TXL_{ratio} = \frac{TXL_{outputs}}{TXL_{total}}$$
$$TXL_{\bar{ratio}} = \frac{TXL_{errors}}{TXL_{total}}$$

In [2]:
def parse_unparse_contracts(txl: str, paths: list) -> Tuple[List[str], List[str], List[str]]:
    """
    Parse and unparse contracts.
    using TXL
    :return:
    """
    contracts_num = 0
    errors = []
    errno = []
    outputs = []
    # for each file in the contracts folder
    # do the command 'txl ?.sol ?.txl'
    # write a progress bar
    tqdm.tqdm.write('Parsing contracts...')
    for path in tqdm.tqdm(paths):
        # get the files in the contracts folder
        for p in os.listdir(path):
            if p.endswith('.sol'):
                ps = os.path.abspath(os.path.join(path, p))
            contracts_num += 1
            # get errors and output from txl
            proc = subprocess.Popen(['txl', txl, ps], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            out, err = proc.communicate()

            # Check return code to see if txl command was successful
            if proc.returncode != 0:
                error_message = err.decode('utf-8') if err else 'Unknown error'
                # Log the error
                # tqdm.tqdm.write(f'Error processing {ps}:\n{error_message}')
                errors.append(ps)
                errno.append(error_message)
                # If you want to raise an exception instead of just logging:
                # raise RuntimeError(f'Error processing {ps}:\n{error_message}')

            # if there is output
            if out:
                output_message = out.decode('utf-8')
                # Uncomment below if you want to print the output
                # tqdm.tqdm.write(f'Output from processing {ps}:\n{output_message}')
                outputs.append(output_message)

    # show progress bar showing the number of files parsed and did not parse
    tqdm.tqdm.write('Parsed: {}'.format(len(outputs)))
    tqdm.tqdm.write('Errors: {}'.format(len(errors)))
    tqdm.tqdm.write('Total: {}'.format(len(outputs) + len(errors)))
    tqdm.tqdm.write('TOTAL CONTRACTS: {}'.format(contracts_num))
    # ratio of files parsed to files not parsed
    tqdm.tqdm.write('Ratio Parsed: {}'.format(len(outputs) / (len(outputs) + len(errors))))
    tqdm.tqdm.write('Ratio ~Parsed: {}'.format(len(errors) / (len(outputs) + len(errors))))
    return outputs, errors, errno

# Parse and Unparse Contracts (get all the contracts)

In [3]:
# get the path to the contracts
contracts_path = [os.path.abspath(os.path.join(os.getcwd(), '..', 'scripts', 'custom_contracts')),
                    os.path.abspath(os.path.join(os.getcwd(), '..', 'scripts', 'gpt_contracts')),
                    os.path.abspath(os.path.join(os.getcwd(), '..', 'scripts', 'github_contracts', 'error_contracts')),
                    os.path.abspath(os.path.join(os.getcwd(), '..', 'scripts', 'github_contracts', 'verified_contracts')),
                  ]

contracts_path

['/Users/sourenakhanzadeh/programming/txl/SolOSphere/scripts/custom_contracts',
 '/Users/sourenakhanzadeh/programming/txl/SolOSphere/scripts/gpt_contracts',
 '/Users/sourenakhanzadeh/programming/txl/SolOSphere/scripts/github_contracts/error_contracts',
 '/Users/sourenakhanzadeh/programming/txl/SolOSphere/scripts/github_contracts/verified_contracts']

In [4]:
# get the path to the txl file
txl_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'SolO', 'txl', 'new_version', 'optimize.txl'))

txl_path

'/Users/sourenakhanzadeh/programming/txl/SolOSphere/SolO/txl/new_version/optimize.txl'

In [5]:
# parse and unparse the contracts
outputs, errors, errno = parse_unparse_contracts(txl_path, contracts_path)

Parsing contracts...


100%|██████████| 4/4 [00:07<00:00,  1.94s/it]

Parsed: 610
Errors: 244
Total: 854
TOTAL CONTRACTS: 854
Ratio Parsed: 0.7142857142857143
Ratio ~Parsed: 0.2857142857142857


In [10]:
# take a look at errors
actual_err = []
for e in errno:
    er = next((line for line in e.split('\n') if '>>>' in line), None)
    if er:
        actual_err.append(er)
    else:
        actual_err.append('Unknown Error')

actual_err

['\tcontract ERC20 >>> ( <<< ) : def transfer ( ',
 '\t0 , 32 ) pop >>> } <<< } function updateExchange ( address ',
 '\t_txData ) ) if gt >>> ( <<< color , 49152 ) { ',
 '\t0 , 32 ) pop >>> } <<< } function updateExchange ( address ',
 '\t, returndatasize ) if iszero >>> ( <<< result ) { revert ( ',
 '\t, off ) if gt >>> ( <<< add ( len , off ',
 '\t0 , 32 ) pop >>> } <<< } function updateExchange ( address ',
 '\t, returndatasize ) if iszero >>> ( <<< result ) { revert ( ',
 '\t0 , 32 ) pop >>> } <<< } function updateExchange ( address ']

In [7]:
nicad_contracts = os.path.abspath(os.path.join(os.getcwd(), '..', 'scripts', 'nicad_contracts'))

nicad_contracts

'/Users/sourenakhanzadeh/programming/txl/SolOSphere/scripts/nicad_contracts'

In [8]:
# parse and unparse the contracts
outputs, errors, errno = parse_unparse_contracts(txl_path, [nicad_contracts])

Parsing contracts...


100%|██████████| 1/1 [00:03<00:00,  3.52s/it]

Parsed: 358
Errors: 9
Total: 367
TOTAL CONTRACTS: 367
Ratio Parsed: 0.9754768392370572
Ratio ~Parsed: 0.02452316076294278


In [9]:
errno

["TXL v10.8b (13.7.22) (c) 1988-2022 Queen's University at Kingston\nCompiling /Users/sourenakhanzadeh/programming/txl/SolOSphere/SolO/txl/new_version/optimize.txl ... \nParsing /Users/sourenakhanzadeh/programming/txl/SolOSphere/scripts/nicad_contracts/Vyper_contract.sol ...\n[/Users/sourenakhanzadeh/programming/txl/SolOSphere/scripts/nicad_contracts/Vyper_contract.sol, /Users/sourenakhanzadeh/programming/txl/SolOSphere/SolO/txl/new_version/optimize.txl] : TXL0192E line 1 of /Users/sourenakhanzadeh/programming/txl/SolOSphere/scripts/nicad_contracts/Vyper_contract.sol - Syntax error at or near:\n\tcontract ERC20 >>> ( <<< ) : def transfer ( \n",
 "TXL v10.8b (13.7.22) (c) 1988-2022 Queen's University at Kingston\nCompiling /Users/sourenakhanzadeh/programming/txl/SolOSphere/SolO/txl/new_version/optimize.txl ... \nParsing /Users/sourenakhanzadeh/programming/txl/SolOSphere/scripts/nicad_contracts/ExchangeV3.sol ...\n[/Users/sourenakhanzadeh/programming/txl/SolOSphere/scripts/nicad_contract